In [8]:
# Cell 1: Imports
import pandas as pd
import numpy as np
from datetime import datetime
import difflib

# If needed for loading raw data
df_raw = pd.read_csv("raw_order_data.csv")


In [3]:
# Cell 2: Reference data lists (for cleaning)
REGIONS = ["Mumbai", "Delhi", "Kolkata", "Chennai", "Bengaluru", "Hyderabad", "Ahmedabad", "Pune", "Lucknow", "Jaipur"]


In [9]:
# Cell 3: Data Cleaning Functions
def clean_quantity(x):
    if pd.isna(x) or str(x).strip()=="" :
        return np.nan, "quantity_missing"
    s = str(x).strip()
    words_map = {"one":1,"two":2,"three":3,"four":4,"five":5}
    if s.lower() in words_map:
        return words_map[s.lower()], "quantity_word_converted"
    try:
        val = int(float(s))
        return val, ""
    except:
        digits = ''.join(ch for ch in s if ch.isdigit() or ch=='-')
        try:
            return int(digits), "quantity_extracted"
        except:
            return np.nan, "quantity_bad"

def clean_unitprice(x):
    if pd.isna(x) or str(x).strip()=="" :
        return np.nan, "unitprice_missing"
    s = str(x).strip().replace(",","")
    try:
        val = int(float(s))
        return val, ""
    except:
        digits = ''.join(ch for ch in s if ch.isdigit())
        try:
            return int(digits), "unitprice_extracted"
        except:
            return np.nan, "unitprice_bad"

def fix_email(e):
    if pd.isna(e) or str(e).strip()=="" :
        return "unknown@example.com", "email_missing"
    s = str(e).strip()
    if "@" not in s or "." not in s.split("@")[-1]:
        if "gnail" in s:
            s = s.replace("gnail", "gmail")
            return s, "email_fixed_gnail"
        if "con" in s and "@" in s:
            s = s.replace("con", "com")
            return s, "email_fixed_con"
        if " " in s:
            s = s.replace(" ", "")+"@mail.com"
            return s, "email_fixed_space"
        return "unknown@example.com", "email_invalid"
    return s, ""

region_map = {r.lower(): r for r in REGIONS}
def normalize_region(r):
    if pd.isna(r) or str(r).strip() == "":
        return "Unknown", "region_missing"
    s = str(r).strip()
    s_lower = s.lower()
    if s_lower in region_map:
        return region_map[s_lower], ""
    for reg in REGIONS:
        if s_lower.startswith(reg[:3].lower()):
            return reg, "region_fixed_prefix"
    # Force closest match from region list regardless of similarity
    close_matches = difflib.get_close_matches(s, REGIONS, n=1, cutoff=0.0)
    if close_matches:
        return close_matches[0], "region_fuzzy_forced"
    return s.title(), "region_titlecased"


def parse_date(d):
    if pd.isna(d) or str(d).strip()=="" :
        return pd.NaT, "date_missing"
    s = str(d).strip()
    for fmt in ("%Y-%m-%d","%d/%m/%Y","%d-%m-%Y","%d %b %Y","%b %d %Y","%Y/%m/%d","%d.%m.%Y",
                "%d %B %Y","%d-%b-%Y","%d %B, %Y","%d %b, %Y"):
        try:
            return pd.to_datetime(datetime.strptime(s, fmt)).date(), ""
        except Exception:
            continue
    try:
        return pd.to_datetime(s, dayfirst=True, errors="coerce").date(), "date_parsed_fallback"
    except:
        return pd.NaT, "date_bad"


In [11]:
# Cell 4: Main Cleaning Dataframe Function
def clean_dataframe(df_raw):
    df = df_raw.copy()
    df.columns = [c.strip() for c in df.columns]

    cleaned_rows = []
    for idx, row in df.iterrows():
        row_notes = []
        orderid = str(row.get("OrderID","")).strip()
        cname = str(row.get("CustomerName","")).strip()
        if cname=="" or cname.lower() in ["unknown","nan"]:
            cname = "Unknown Customer"
            row_notes.append("name_missing")
        email, enote = fix_email(row.get("Email",""))
        if enote: row_notes.append(enote)
        region, rnote = normalize_region(row.get("Region",""))
        if rnote: row_notes.append(rnote)
        product = str(row.get("Product","")).strip()
        product = product.replace("(used)","").replace("- refurbished","").replace("  "," ").strip()
        qty_val, qnote = clean_quantity(row.get("Quantity",""))
        if qnote: row_notes.append(qnote)
        up_val, unote = clean_unitprice(row.get("UnitPrice",""))
        if unote: row_notes.append(unote)
        od_val, dnote = parse_date(row.get("OrderDate",""))
        if dnote: row_notes.append(dnote)
        delivery = str(row.get("DeliveryStatus","")).strip().title() if str(row.get("DeliveryStatus","")).strip() else "Unknown"
        payment = str(row.get("PaymentStatus","")).strip().title() if str(row.get("PaymentStatus","")).strip() else "Unknown"
        t_raw = row.get("Total","")
        try:
            t_val = int(float(str(t_raw))) if str(t_raw).strip()!="" else np.nan
        except:
            digits = ''.join(ch for ch in str(t_raw) if ch.isdigit())
            t_val = int(digits) if digits!="" else np.nan
        expected_total = None
        if not pd.isna(qty_val) and not pd.isna(up_val):
            expected_total = int(qty_val * up_val)
            if pd.isna(t_val):
                t_val = expected_total
                row_notes.append("total_filled_from_calc")
            else:
                if abs(t_val - expected_total) > max(1, 0.05 * expected_total):
                    row_notes.append(f"total_mismatch_expected_{expected_total}")
        if not pd.isna(qty_val) and qty_val < 0:
            row_notes.append("quantity_negative_marked_return")
        cleaned = {
            "OrderID": orderid,
            "CustomerName": cname,
            "Email": email,
            "Region": region,
            "Product": product,
            "Quantity": int(qty_val) if not pd.isna(qty_val) else np.nan,
            "UnitPrice": int(up_val) if not pd.isna(up_val) else np.nan,
            "Total": int(t_val) if not pd.isna(t_val) else np.nan,
            "ExpectedTotal": int(expected_total) if expected_total is not None else np.nan,
            "OrderDate": pd.to_datetime(od_val) if not pd.isna(od_val) else pd.NaT,
            "DeliveryStatus": delivery,
            "PaymentStatus": payment,
            "ValidationNotes": "; ".join(row_notes) if row_notes else ""
        }
        cleaned_rows.append(cleaned)

    df_clean = pd.DataFrame(cleaned_rows)
    dup_mask = df_clean.duplicated(subset=["OrderID"], keep="first")
    if dup_mask.any():
        df_clean.loc[dup_mask, "ValidationNotes"] = df_clean.loc[dup_mask, "ValidationNotes"].astype(str) + "; duplicate_orderid_dropped"
    df_clean = df_clean.drop_duplicates(subset=["OrderID"], keep="first").reset_index(drop=True)
    return df_clean


In [12]:
# Cell 5: Save to Excel or CSV
def save_to_excel(df_raw, df_cleaned, path="OrderCleanser_Dataset.xlsx"):
    try:
        with pd.ExcelWriter(path, engine="openpyxl") as writer:
            df_raw.to_excel(writer, sheet_name="Raw_Data", index=False)
            df_cleaned.to_excel(writer, sheet_name="Cleaned_Data", index=False)
        return path
    except Exception as e:
        with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
            df_raw.to_excel(writer, sheet_name="Raw_Data", index=False)
            df_cleaned.to_excel(writer, sheet_name="Cleaned_Data", index=False)
        return path


In [13]:
# Cell 6: Load raw data, clean and save
# Assuming raw data is read from CSV or directly passed as df_raw

# Example using raw CSV file:
# df_raw = pd.read_csv("raw_order_data.csv")

# If df_raw is already in memory, skip loading step

df_clean = clean_dataframe(df_raw)
print("Cleaned data sample:")
display(df_clean.head(10))

output_path = save_to_excel(df_raw, df_clean, path="OrderCleanser_Dataset.xlsx")
print(f"Cleaned and raw data saved to: {output_path}")


Cleaned data sample:


,OrderID,CustomerName,Email,Region,Product,Quantity,UnitPrice,Total,ExpectedTotal,OrderDate,DeliveryStatus,PaymentStatus,ValidationNotes
0,ORD1082,Rahul Kaur,rahul.kaur@hotmail.com,Lucknow,Ink Cartridge,1,889.0,889.0,889.0,2015-10-13,Cancelled,Unpaid,
1,ORD1219,Suresh Nair,suresh.nair@gmail.com,Pune,Ink Cartridge,2,785.0,1570.0,1570.0,2017-09-13,Pending,Cod,
2,ORD1056,Rita Verma,rita.verma@yahoo.com,Delhi,Speaker,1,1765.0,1765.0,1765.0,2015-12-15,Delayed,Cod,
3,ORD1495,Ramesh Gupta,unknown@example.com,Chennai,Keyboard,1,740.0,740.0,740.0,2018-11-14,Delayed,Unpaid,email_missing
4,ORD1265,Vijay Sharma,vijay.sharma@mail.com,Lucknow,Tablet,2,15412.0,15412.0,30824.0,2018-03-19,Delivered,Refunded,total_mismatch_expected_30824
5,ORD1156,Neha Singh,neha.singh@hotmail.com,Bengaluru,Headphones,1,1645.0,1645.0,1645.0,2018-05-06,Out For Delivery,Cod,
6,ORD1484,Meena Iyer,meena.iyer@hotmail.com,Bengaluru,Charger,2,NaN,910.0,NaN,2023-07-19,Pending,Paid,quantity_word_converted; unitprice_missing
7,ORD1322,Geeta Kumar,geeta.kumar@yahoo.com,Lucknow,Phone Case,1,290.0,290.0,290.0,2021-04-16,Delayed,Cod,
8,ORD1583,Imran Khan,imran.khan@yahoo.com,Pune,Microphone,1,2967.0,2967.0,2967.0,2022-07-18,Out For Delivery,Unpaid,
9,ORD1292,Latika Singh,latika.singh@hotmail.com,Kolkata,Ink Cartridge,1,767.0,767.0,767.0,2020-08-27,Cancelled,Paid,


Cleaned and raw data saved to: OrderCleanser_Dataset.xlsx
